In [1]:
import csv
import json
import sys
import os
sys.path.append('/root/pva-faster-rcnn/lib')
sys.path.append('/root/pva-faster-rcnn/lib/datasets')
import time
from datasets.imdb import imdb
import datasets.ds_utils as ds_utils
import numpy as np
import scipy.sparse
import scipy.io as sio
import utils.cython_bbox
import cPickle
import subprocess
import uuid
from voc_eval import voc_eval
from fast_rcnn.config import cfg
import json
from os import listdir
from os.path import isfile, join
from fast_rcnn.test import im_detect
from fast_rcnn.nms_wrapper import nms
from utils.timer import Timer
import glob
import cv2
from datasets.config import CLASS_SETS
from natsort import natsorted



import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
%matplotlib inline


In [ ]:
jsonInput = open('./25typesTrain.json','r')
annos_25 = json.load(jsonInput)

image_path = '/root/data/data-openImages_v4/train'
set_num = 0
img_pattern = "{}/*.jpg".format(image_path)       
img_paths = natsorted(glob.glob(img_pattern))

target_imgs_25 = [os.path.basename(i)[:-4] for i in img_paths[:500000]]
#print target_imgs
  
    
    
jsonInput.close()
#print original_classes

In [ ]:
jsonInput = open('/root/data/data-airport/annotations.json','r')
annos = json.load(jsonInput)
image_path = '/root/data/data-airport/images'
set_num = 0


img_pattern = "{}/set0{}/V000/set0{}_V*.jpg".format(image_path,set_num,set_num)
print img_pattern   
img_paths = natsorted(glob.glob(img_pattern))
target_imgs = [os.path.basename(i)[:-4] for i in img_paths]

for index in target_imgs: 
    set_nuam, v_num, frame = index.split("_")
    print frame
    bboxes = annos[str(set_num)].get(frame, {}).values()
    bboxes = [bbox for bbox in bboxes if bbox['outside']==0 and bbox['occluded']==0]
    print bboxes

    jsonInput.close()

In [ ]:
trainingConfig = {    
    
    

    "labelList":['Backpack','Baseball bat','Baseball glove', 'Bag','Man','Woman',
             'Bicycle','Rifle','Handgun','Shotgun','Suit','Gun',
             'Dress','Jacket','Skirt','Shorts','Sports Uniform','Shirt','Trousers',
             'Mini skirt','Goggles','Glasses','Headphones','Sunglasses',
             'Hat','Helmet','Sun hat','Football helmet','Cowboy hat','Sombrero','Roller skates',
             'Boot','Handbag','Backpack','Suitcase','Plastic bag','Briefcase','Wheelchair',
             'Umbrella','Scarf','Tie'],

    u'mapperList':{'Rifle':'Gun','Handgun':'Gun','Shotgun':'Gun','Boy':'Man','Girl':'Woman',
              'Mini skirt':'Skirt','Goggles':'Glasses','Sunglasses':'Glasses','Sun hat':'Hat',
              'Football helmet':'Helmet','Cowboy hat':'Hat','Sombrero':'Hat','Handbag':'Bag',
              'Plastic bag':'Bag','Briefcase':'Bag'}
    'gpu_id':'2'
    
}


with open('test.json', "w") as f:
    json.dump(trainingConfig,f)


In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print '\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix),'\r';
    # Print New Line on Complete
    if iteration == total: 
        print()
        
from time import sleep

# A List of Items
items = list(range(0, 57))
l = len(items)

# Initial call to print 0% progress
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for i, item in enumerate(items):
    # Do stuff...
    sleep(0.1)
    # Update Progress Bar
    printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

In [336]:
def get_data_map(path="/root/data", prefix="data-"):
    data_map = {} 
    data_paths = glob.glob("{}/{}*".format(path, prefix))
    for data_path in data_paths:
        name = os.path.basename(data_path)[5:]
        data_map[name] = data_path
    return data_map    

data_map = get_data_map()
data_names = data_map.keys()
    
    

def has_data(name):
    return name in data_names

def load_meta(meta_path):
    if os.path.exists(meta_path):
        meta = json.load(open(meta_path))
    else:
       
        meta = {"format":"jpg"}
        meta["train"] = {"start":None, "end":None, "stride":1, "sets":[0]}
        meta["test"] = {"start":None, "end":None, "stride":30, "sets":[1]}
        print("Meta data path: {} does not exist. Use Default meta data".format(meta_path))
    return meta

class datasetTesting(imdb):
    
    
            
    def loadMapper(self,mapperPath,mapperList):
        mapper = {}
        reverseMapper = {}
        f = open(mapperPath,'r')
        for i in csv.reader(f):
            key = i[0]
            if mapperList.has_key(i[1]):
                val = mapperList.get(i[1])
            else:
                val = i[1]
            mapper[key] = val
            reverseMapper[val] = key
        f.close()
        return mapper,reverseMapper  
    
    def getAnnotation(self,labelList,mapperList,sets='train'):
        
        mapperPath = '/root/data/data-openImages_v4/class-descriptions-boxable.csv'
        mapper,reverseMapper = self.loadMapper(mapperPath,mapperList)
        method = ['freeform ','xclick']
        bboxGTPath = '/root/data/data-openImages_v4/{}-annotations-bbox.csv'.format(sets)
        seq = [reverseMapper.get(i) for i in labelList]
        f = open(bboxGTPath, 'r')
        annotations = {}
        mappedClass = {}
        for row in csv.reader(f):
            if row[1] not in method:
                continue
            if row[2] not in seq:
                continue
            if os.path.isfile(os.path.join('/root/data/data-openImages_v4/{}'.format(sets),row[0]+'.jpg')):
                if annotations.has_key(row[0]):
                    annotations[row[0]] += [row[2:]]
                    mappedClass[row[0]] += [[mapper.get(row[2])]+row[3:]]
                else:
                    annotations[row[0]] = [row[2:]]
                    mappedClass[row[0]] = [[mapper.get(row[2])]+row[3:]]
        f.close()
        return annotations,mappedClass
    
    def parseConfig(self):
        jsonInput = open(self.configPath,'r')
        annos = json.load(jsonInput)
        CLS_mapper = annos.get("CLS_mapper")
        labelList = annos.get("labelList")
        jsonInput.close()
        return CLS_mapper,labelList
    

    
    def __init__(self, datasetName, annotationPath,configPath):
        
        name="openImages_v4"
        #FOR DEBUGGING
        self.debugging = True
        
        self.configPath = configPath
        assert os.path.exists(configPath), \
                'Config path does not exist.: {}'.format(configPath)
        self.annotationPath = annotationPath
        #os.path.join(self._data_path, "annotations.json")         
        assert os.path.exists(annotationPath), \
                'Annotation path does not exist.: {}'.format(annotationPath)

        imdb.__init__(self,name)
        assert data_map.has_key(name),\
        'The {} dataset does not exist. The available dataset are: {}'.format(name, data_map.keys())
        
        
        CLS_mapper,labelList =  self.parseConfig()
        print labelList
        self._classes = labelList
        print self._classes
        self.CLS_mapper = CLS_mapper
        
        namedAnnotation,annotation = self.getAnnotation(self._classes,self.CLS_mapper)
        
        self._annotation = annotation
            
        self._data_path = data_map[name]  
        assert os.path.exists(self._data_path), \
        'Path does not exist: {}'.format(self._data_path)
        
        
        self._class_to_ind = dict(zip(self.classes, xrange(self.num_classes)))
        

        
        self.original_classes = self.get_original_classes()
        
        
        
        
        meta_data_path = os.path.join(self._data_path, "meta.json")         
       
            
        self._meta = load_meta(meta_data_path)
        

        self._image_ext = self._meta["format"]    
        self._image_ext = '.jpg'
        self._image_index = self._get_image_index()

        
    def get_original_classes(self):
        original_classes = set()
        for bboxes in self._annotation.values():
            original_classes.add(bboxes[0][0])
        return original_classes    

In [337]:
a = datasetTesting("testing","/root/pva-faster-rcnn/25typesTrain.json","/root/pva-faster-rcnn/test.json")

None
None


AttributeError: 'NoneType' object has no attribute 'has_key'

In [10]:
a = {"a":1}
print a.get('a')

1


In [7]:
print sum([14339,450,15425,4307,15910,5580,2919])

58930
